In [2]:
# from app.core.model_builder import ModelBuilder
from sixgill.pipesim import Model
from sixgill.definitions import ModelComponents, Parameters
from app.project import get_string_values_from_class, generate_dict_from_class
from sixgill.core.model_context import ContextError
from sixgill.core.mapping import ParameterError
import pandas as pd
import logging

logger = logging.getLogger(__name__)

In [5]:
file = r"F:\Project\AET01254 ADNOC Gas Feasibility of BFW control valve\09. Documents, Drawings\9.71 Project Management\09.71.01 Current documents\5. Hydraulic Report\Rev 0\Pipesim Models\Habshan 5 BFW Network - 93%.pips"

In [6]:
model = Model.open(file)

INFO:manta.server.manager:Starting PIPESIM server on thread ID: 4636
INFO:manta.server.manager:Python toolkit license is checked out successfully
INFO:manta.server.manager:Waiting for PIPESIM server to start.
INFO:sixgill.core.metadata:Using cached metadata for 'http://localhost:60521/api/metadata#'
